# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  May 31 2022  9:24AM  238367       102  PBR-051822-01   
1  May 31 2022  9:21AM  238373        19        8086057   
2  May 31 2022  9:21AM  238373        19        8086559   
3  May 31 2022  9:21AM  238373        19        8086561   
4  May 31 2022  9:21AM  238373        19        8086564   
5  May 31 2022  9:21AM  238369        10     0085721493   
6  May 31 2022  9:21AM  238369        10     0085721482   
7  May 31 2022  9:21AM  238369        10     0085721488   
8  May 31 2022  9:21AM  238369        10     0085721486   
9  May 31 2022  9:21AM  238369        10     0085721505   

                   Customer ShipmentStatus  
0  Senseonics, Incorporated       Released  
1            Graystone, LLC       Released  
2            Graystone, LLC       Released  
3            Graystone, LLC       Released  
4            Graystone, LLC       Released  
5         ISDIN Corporation       Released  
6         ISDIN Corporation       Released  
7         ISDIN Corporation       Released  
8         ISDIN Corporation       Released  
9         ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
24  238367       Released          1
25  238369       Released         48
26  238370       Released          3
27  238372       Released          3
28  238373       Released          4

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
238367                NaN       1.0
238369                NaN      48.0
238370                NaN       3.0
238372                NaN       3.0
238373                NaN       4.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
238334                0.0      14.0
238335                1.0       0.0
238338                1.0       0.0
238339                0.0       1.0
238341                0.0      35.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
238334                  0        14
238335                  1         0
238338                  1         0
238339                  0         1
238341                  0        35

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               238334          0        14
1               238335          1         0
2               238338          1         0
3               238339          0         1
4               238341          0        35

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               238334                 14
1               238335         1         
2               238338         1         
3               238339                  1
4               238341                 35

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0    May 31 2022  9:24AM  238367       102  Senseonics, Incorporated
1    May 31 2022  9:21AM  238373        19            Graystone, LLC
5    May 31 2022  9:21AM  238369        10         ISDIN Corporation
53   May 31 2022  9:19AM  238372        10         ISDIN Corporation
56   May 31 2022  9:18AM  238370        10               Emerginnova
59   May 31 2022  9:12AM  238362        10         ISDIN Corporation
100  May 31 2022  9:12AM  238366        22         NBTY Global, Inc.
101  May 31 2022  9:12AM  238365        10  Senseonics, Incorporated
102  May 31 2022  9:11AM  238364        22         NBTY Global, Inc.
103  May 31 2022  9:11AM  238363        12                 Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Executing  \
0  May 31 2022  9:24AM  238367       102  Senseonics, Incorporated             
1  May 31 2022  9:21AM  238373        19            Graystone, LLC             
2  May 31 2022  9:21AM  238369        10         ISDIN Corporation             
3  May 31 2022  9:19AM  238372        10         ISDIN Corporation             
4  May 31 2022  9:18AM  238370        10               Emerginnova             
5  May 31 2022  9:12AM  238362        10         ISDIN Corporation             
6  May 31 2022  9:12AM  238366        22         NBTY Global, Inc.             
7  May 31 2022  9:12AM  238365        10  Senseonics, Incorporated             
8  May 31 2022  9:11AM  238364        22         NBTY Global, Inc.             
9  May 31 2022  9:11AM  238363        12                 Hush Hush             

  Released  
0        1  
1        4  
2       48  
3        3  
4        3  
5       41  
6        1  
7        1  
8        1  
9        5

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer Released  \
0  May 31 2022  9:24AM  238367       102  Senseonics, Incorporated        1   
1  May 31 2022  9:21AM  238373        19            Graystone, LLC        4   
2  May 31 2022  9:21AM  238369        10         ISDIN Corporation       48   
3  May 31 2022  9:19AM  238372        10         ISDIN Corporation        3   
4  May 31 2022  9:18AM  238370        10               Emerginnova        3   
5  May 31 2022  9:12AM  238362        10         ISDIN Corporation       41   
6  May 31 2022  9:12AM  238366        22         NBTY Global, Inc.        1   
7  May 31 2022  9:12AM  238365        10  Senseonics, Incorporated        1   
8  May 31 2022  9:11AM  238364        22         NBTY Global, Inc.        1   
9  May 31 2022  9:11AM  238363        12                 Hush Hush        5   

  Executing  
0            
1            
2            
3            
4            
5            
6            
7            
8            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  May 31 2022  9:24AM  238367       102  Senseonics, Incorporated        1   
1  May 31 2022  9:21AM  238373        19            Graystone, LLC        4   
2  May 31 2022  9:21AM  238369        10         ISDIN Corporation       48   
3  May 31 2022  9:19AM  238372        10         ISDIN Corporation        3   
4  May 31 2022  9:18AM  238370        10               Emerginnova        3   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                  Customer  Released  \
0  May 31 2022  9:24AM  238367       102  Senseonics, Incorporated       1.0   
1  May 31 2022  9:21AM  238373        19            Graystone, LLC       4.0   
2  May 31 2022  9:21AM  238369        10         ISDIN Corporation      48.0   
3  May 31 2022  9:19AM  238372        10         ISDIN Corporation       3.0   
4  May 31 2022  9:18AM  238370        10               Emerginnova       3.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  May 31 2022  9:24AM  238367       102  Senseonics, Incorporated       1.0   
1  May 31 2022  9:21AM  238373        19            Graystone, LLC       4.0   
2  May 31 2022  9:21AM  238369        10         ISDIN Corporation      48.0   
3  May 31 2022  9:19AM  238372        10         ISDIN Corporation       3.0   
4  May 31 2022  9:18AM  238370        10               Emerginnova       3.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         3098678     286.0        2.0
102         238367       1.0        0.0
12         1191736      78.0        0.0
16          238338       0.0        1.0
19          715071      12.0        4.0
20          476669      14.0        1.0
22          476730       2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  3098678     286.0        2.0
1       102   238367       1.0        0.0
2        12  1191736      78.0        0.0
3        16   238338       0.0        1.0
4        19   715071      12.0        4.0
5        20   476669      14.0        1.0
6        22   476730       2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     286.0        2.0
1       102       1.0        0.0
2        12      78.0        0.0
3        16       0.0        1.0
4        19      12.0        4.0
5        20      14.0        1.0
6        22       2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  286.0
1       102  Released    1.0
2        12  Released   78.0
3        16  Released    0.0
4        19  Released   12.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102    12   16    19    20   22
Status                                           
Executing    2.0  0.0   0.0  1.0   4.0   1.0  0.0
Released   286.0  1.0  78.0  0.0  12.0  14.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102    12   16    19    20   22
0          Executing    2.0  0.0   0.0  1.0   4.0   1.0  0.0
1           Released  286.0  1.0  78.0  0.0  12.0  14.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102    12   16    19    20   22
0  Executing    2.0  0.0   0.0  1.0   4.0   1.0  0.0
1   Released  286.0  1.0  78.0  0.0  12.0  14.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()